I'll be using CNN articles as dataset. These articles are stored in multiple files along with their highlights. These highlight can be used to gauge the summary output. 

In [16]:
from tqdm.notebook import tqdm 

Let's try reading the file content 

In [17]:
def read_file(filename):
    file = open(filename, encoding= 'UTF-8')
    text = file.read()
    file.close()
    return text

text = read_file('/Users/rohankilledar/Documents/MSc Artificial Intelligence/repos/summarisation/cnn/stories/0a0a4c90d59df9e36ffec4ba306b4f20f3ba4acb.story')
#text.split("\n")



It seems the we need to do some cleaning before we start working on the file.

In [18]:
import re
from nltk.corpus import stopwords
import string
import nltk
nltk.download('stopwords')
en_stopwords = stopwords.words('english')

def read_file(filename):
    file = open(filename, encoding= 'UTF-8')
    text = file.read()
    file.close()
    # cnn_indx = text.find("(CNN)")
    # if cnn_indx >=0:
    #     text = text[cnn_indx+len("(CNN)"):]
        
    #text = re.sub("\n"," ",text).lower()

    #text.split("\n")
    # cleaned = list()
    # table = str.maketrans('','',string.punctuation)

    # for line in text:
       
    #     line = line.split(" ")

    #     line = [w.translate(table) for w in line]
        
    #     line = [w for w in line if w.isalpha()]
    #     cleaned.append(' '.join(line))

    # text = [sent for sent in cleaned if len(sent)>0]

    return text

article = read_file('/Users/rohankilledar/Documents/MSc Artificial Intelligence/repos/summarisation/cnn/stories/0a0a4c90d59df9e36ffec4ba306b4f20f3ba4acb.story')

# t = article.split("\n")
# cleaned = list()
# table = str.maketrans('','',string.punctuation)
# for w in t:
#     if len(w)>0:
#         w = w.split(" ")
#         w = [word.translate(table) for word in w]
#         w = [word for word in w if word.isalpha()]
#         cleaned.append(" ".join(w))
# print(cleaned)
print(article)


(CNN) -- Can a movie actually convince you to support torture? Can a movie really persuade you that "fracking" -- a process used to drill for natural gas -- is a danger to the environment? Can a movie truly cause you to view certain minority groups in a negative light?

Some scoff at the notion that movies do anything more than entertain. They are wrong. Sure, it's unlikely that one movie alone will change your views on issues of magnitude. But a movie (or TV show) can begin your "education" or "miseducation" on a topic. And for those already agreeing with the film's thesis, it can further entrench your views.

Anyone who doubts the potential influence that movies can have on public opinion need to look no further than two films that are causing an uproar even before they have opened nationwide. They present hot button issues that manage to fire up people from the left and right.

The first, "Zero Dark Thirty," is about the pursuit and killing of Osama bin Laden, which features scenes 

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rohankilledar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
line  = article.split("\n")

# ex = re.split(r"(CNN)*-- ", article)
# print(ex)

Now we need to split the article and the hightlights

In [20]:
def split_text(article):
    indx = article.index('@highlight')
    story = article[:indx]
    highlight = article[indx:].split('@highlight')
    highlight = [h.strip() for h in highlight if len(h)>0]
    return story,highlight

st,hi = split_text(article)
#print(st)
#print(hi)

one of the dataset file seems to be usable now. Need to load all of the stories together using these given functions

In [37]:
from os import listdir

def read_all(folder):
    dataset = list()

    for file in tqdm(listdir(folder)):
        filename = folder + '/' + file
        article = read_file(filename)
        story,highlight = split_text(article)

        dataset.append({'story':story, 'highlight':highlight})
    
    return dataset

folder = "/Users/rohankilledar/Documents/MSc Artificial Intelligence/repos/summarisation/cnn/stories"
dataset = read_all(folder)
print("total stories in dataset: "+ str(len(dataset)))

  0%|          | 0/92579 [00:00<?, ?it/s]

total stories in dataset: 92579


In [39]:
example = dataset[1]['highlight']
example


['Zimmerman posts $5,000 bail; he was accused of throwing a bottle at a girlfriend',
 '"He hasn\'t been very lucky with the ladies," attorney says of Zimmerman',
 "He became a national figure after being charged, then acquitted in Trayvon Martin's death"]

Data Cleaning

In [40]:
import re
def clean_data(data):
    cleaned = list()
    table = str.maketrans('', '', string.punctuation)
    for line in data:
        #line = re.split(r"(CNN)+ --",line)

        dash_indx = line.find("--")
        if dash_indx >=0:
            line = line[dash_indx+len("--"):]

        #line = line.trim()
        # tokenize on white space
        line = line.split()
        # convert to lower case
        line = [word.lower() for word in line]
        # remove punctuation from each token
        line = [w.translate(table) for w in line]
        # remove tokens with numbers in them
        line = [word for word in line if word.isalpha()]
        # store as string
        cleaned.append(' '.join(line))
	# remove empty strings
    cleaned = [c for c in cleaned if len(c) > 0]
    return cleaned


#dataset[0]['story'] = clean_data(dataset[0]['story'].split('\n'))
#dataset[0]['highlight'] = clean_data(dataset[0]['highlight'])
#     #text = re.sub("\n"," ",text).lower()
# cleaned_dataset = dataset.copy()
for i in tqdm(range(len(dataset))):

    dataset[i]['story']= clean_data(dataset[i]['story'].split('\n'))
    dataset[i]['highlight'] = clean_data(dataset[i]['highlight'])
    

  0%|          | 0/92579 [00:00<?, ?it/s]

In [46]:
dataset[0]['highlight']

['usbased scientists say their data points toward the existence of the higgs boson',
 'finding the higgs boson would help explain the origin of mass',
 'but the research at the tevatron collider doesnt provide a conclusive answer',
 'attention now turns to a seminar wednesday on data from the large hadron collider']

Saving the dataset

In [42]:
# save to file
from pickle import dump
dump(dataset, open('cnn_dataset.pkl', 'wb'))

In [43]:
import csv
csv_columns = ['story','highlight']
csv_file = "cnn.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in dataset:
            writer.writerow(data)
except IOError:
    print("I/O error")

In [44]:
import pandas as pd

dataset_pd = pd.read_csv("cnn.csv")

In [45]:
dataset_pd.head()

,story,highlight
0,['at the start of a big week for the higgs bos...,['usbased scientists say their data points tow...
1,['acquitted by a florida jury over the death o...,['zimmerman posts bail he was accused of throw...
2,['zlatan ibrahimovic scored his third goal in ...,['barcelona move three points clear of real ma...
3,['nobel laureate norman e borlaug an agricultu...,['borlaug died at the age of from complication...
4,['cnnlouisiana gov bobby jindal on monday stoo...,['louisiana gov bobby jindal decried nogo zone...
